<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_conv1D_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

Everst_data=pd.read_csv('average_premium.csv')
data=Everst_data
data['Attained_age']=data['Age']+data['Duration']
data['Premium_rate']=data['BasePremiumAmount']/data['BaseNARAmount']*10000
#exclude_cols = ['Gender','RiskClass']
Coulmns_to_keep=['Age','Premium_rate','Gender','RiskClass','Duration']
#data=data[data['SingleOrJointType']=="Single"]
#data=data[data['Gender']=="Male"]
#data=data[data['RiskClass']=="NS1"]
data=data[Coulmns_to_keep]
data=data[(data['RiskClass']!="SS1") & (data['RiskClass']!="SS2")& (data['RiskClass']!="SP1")]
data['index']=data.index
numerical_columns = data.select_dtypes(include=['number']).columns
categorical_columns = data.select_dtypes(include=['object']).columns
data_copy=data.head(8657*5).copy()
data=data.head(8657*5)

<ipython-input-54-90565239e93f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index']=data.index


In [55]:
# Create a multi-index dataframe using Age, Duration, and a combined Gender and RiskClass column

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
data['RiskClass'] = data['RiskClass'].replace({'N+1': 'NP2', 'NP1': 'NP2'}).replace({'NS3': 'NS2', 'NS2': 'NS2', 'NR1': 'NS2', 'NS1' : 'NS2'})
print(data)
print(data['RiskClass'].unique())
multi_index_df = data.set_index(['Age', 'Duration', data['Gender'] + '_' + data['RiskClass']])

# Convert the multi-index dataframe to a regular dataframe
result_df = multi_index_df.reset_index().drop(['Gender', 'RiskClass'], axis=1)
print(result_df)
result_df['level_2'].unique()

       Age  Premium_rate  Gender RiskClass  Duration  index
0       52     32.884713  Female       NP2         8      0
1       25      7.822222    Male       NP2         7      1
2       20      3.640212  Female       NS2         8      2
4       38     11.758730    Male       NP2         8      4
5       30      5.480635    Male       NS2         8      5
...    ...           ...     ...       ...       ...    ...
47373   23      6.720635    Male       NS2         3  47373
47374   29      3.841270  Female       NS2         4  47374
47375   44     11.541446    Male       NP2         4  47375
47376   36      6.016629    Male       NS2         4  47376
47378   33      4.118519  Female       NS2         4  47378

[43285 rows x 6 columns]
['NP2' 'NS2']
       Age  Duration     level_2  Premium_rate  index
0       52         8  Female_NP2     32.884713      0
1       25         7    Male_NP2      7.822222      1
2       20         8  Female_NS2      3.640212      2
3       38         8    

array(['Female_NP2', 'Male_NP2', 'Female_NS2', 'Male_NS2'], dtype=object)

In [56]:
import numpy as np
import pandas as pd

# Before removing NAs
print("Before removing NAs:")
print(result_df.describe(include='all'))
print(result_df.count())

# Drop rows with missing values
result_df_no_na = result_df.dropna()

# After removing NAs
print("\nAfter removing NAs:")
print(result_df_no_na.describe(include='all'))
print(result_df_no_na.count())


Before removing NAs:
                 Age      Duration     level_2  Premium_rate         index
count   43285.000000  43285.000000       43285  43285.000000  43285.000000
unique           NaN           NaN           4           NaN           NaN
top              NaN           NaN  Female_NP2           NaN           NaN
freq             NaN           NaN       13545           NaN           NaN
mean       37.263994      4.845535         NaN     13.024018  23686.246436
std        11.245521      2.555028         NaN     22.319405  13685.301612
min        18.000000      1.000000         NaN     -9.301587      0.000000
25%        28.000000      3.000000         NaN      4.137566  11843.000000
50%        36.000000      5.000000         NaN      7.228571  23676.000000
75%        45.000000      7.000000         NaN     13.192240  35539.000000
max        70.000000     10.000000         NaN   1105.705706  47378.000000
Age             43285
Duration        43285
level_2         43285
Premium_rate 

In [57]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense, MaxPooling1D

In [74]:
# prompt: for the dsata  result_df_no_na one hot encode the level_2

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assuming 'result_df_no_na' is your DataFrame from the preceding code

# Create a OneHotEncoder object
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit the encoder to the 'level_2' column and transform it
encoded_data = encoder.fit_transform(result_df_no_na[['level_2']])

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['level_2']))

# Concatenate the encoded DataFrame with the original DataFrame
result_df_encoded = pd.concat([result_df_no_na, encoded_df], axis=1)

# Optionally, drop the original 'level_2' column
result_df_encoded = result_df_encoded.drop('level_2', axis=1)

result_df_encoded.drop(['index'], axis=1,inplace=True)
result_df_encoded

,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2
0,52,8,32.884713,1.0,0.0,0.0,0.0
1,25,7,7.822222,0.0,0.0,1.0,0.0
2,20,8,3.640212,0.0,1.0,0.0,0.0
3,38,8,11.758730,0.0,0.0,1.0,0.0
4,30,8,5.480635,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
43280,23,3,6.720635,0.0,0.0,0.0,1.0
43281,29,4,3.841270,0.0,1.0,0.0,0.0
43282,44,4,11.541446,0.0,0.0,1.0,0.0
43283,36,4,6.016629,0.0,0.0,0.0,1.0


In [ ]:
#test_df_with_anomalies = result_df_encoded.iloc[X_test.index].copy()  # X_test is a numpy array and does not have an index
# Reshape the data to 3D for CNN input
data_array = result_df_encoded.values  # Extract the underlying NumPy array
data_reshaped = data_array.reshape(-1, 7, 1)

# Split the data into training and test sets
X_train, X_test = train_test_split(data_reshaped, test_size=0.2, random_state=42)
train_indices, test_indices = train_test_split(result_df_encoded.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2])
input_layer = Input(shape=input_shape)

conv_layer = Conv1D(32, kernel_size=3, activation="relu")(input_layer)
conv_layer = MaxPooling1D(pool_size=1)(conv_layer)
conv_layer = Flatten()(conv_layer)

dense_layer = Dense(10, activation="relu")(conv_layer)
output_layer = Dense(7, activation="linear")(dense_layer)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=50,
                        batch_size=32,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train.reshape(-1, 7) - pred_train, 2), axis=1)

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test.reshape(-1, 7) - pred_test, 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 95)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")


Epoch 1/50
974/974 [==============================] - 5s 4ms/step - loss: 23.1841 - val_loss: 0.6299
Epoch 2/50
974/974 [==============================] - 2s 2ms/step - loss: 0.4606 - val_loss: 0.3347
Epoch 3/50
974/974 [==============================] - 2s 2ms/step - loss: 0.2031 - val_loss: 0.1195
Epoch 4/50
974/974 [==============================] - 3s 3ms/step - loss: 0.0901 - val_loss: 0.0776
Epoch 5/50
974/974 [==============================] - 3s 3ms/step - loss: 0.0772 - val_loss: 0.0739
Epoch 6/50
974/974 [==============================] - 2s 2ms/step - loss: 0.0768 - val_loss: 0.0837
Epoch 7/50
974/974 [==============================] - 2s 2ms/step - loss: 0.1163 - val_loss: 0.6329
Epoch 8/50
974/974 [==============================] - 2s 2ms/step - loss: 0.0770 - val_loss: 0.0710
Epoch 9/50
974/974 [==============================] - 2s 2ms/step - loss: 0.0707 - val_loss: 0.0700
Epoch 10/50
974/974 [==============================] - 3s 3ms/step - loss: 0.0848 - val_loss: 0.069

In [ ]:
# Create a copy of the original test DataFrame
test_df_with_anomalies = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies['anomaly'] = anomalies

print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies

Threshold: 0.14280066700792216
Number of anomalies in the test set: 449


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
42657,19,3,3.182540,1.0,0.0,0.0,0.0,False
38367,30,5,4.298786,0.0,0.0,1.0,0.0,False
17831,40,3,8.730159,0.0,0.0,1.0,0.0,True
7236,45,3,10.587302,0.0,0.0,0.0,1.0,False
19263,35,3,3.619048,0.0,1.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...
2312,25,9,8.644444,0.0,0.0,0.0,1.0,False
16552,31,1,23.777778,1.0,0.0,0.0,0.0,False
22914,27,3,7.111111,0.0,0.0,0.0,1.0,False
23316,36,4,5.140317,1.0,0.0,0.0,0.0,False


In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 402


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
42657,19,3,3.182540,1.0,0.0,0.0,0.0,False
38367,30,5,4.298786,0.0,0.0,1.0,0.0,False
17831,40,3,8.730159,0.0,0.0,1.0,0.0,False
7236,45,3,10.587302,0.0,0.0,0.0,1.0,False
19263,35,3,3.619048,0.0,1.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...
2312,25,9,8.644444,0.0,0.0,0.0,1.0,False
16552,31,1,23.777778,1.0,0.0,0.0,0.0,False
22914,27,3,7.111111,0.0,0.0,0.0,1.0,False
23316,36,4,5.140317,1.0,0.0,0.0,0.0,False


In [80]:
result_df_encoded=result_df_encoded.sort_values(['Age','Duration','level_2_Female_NP2','level_2_Female_NS2','level_2_Male_NP2','level_2_Male_NS2'])
result_df_encoded=result_df_encoded.head((43285//60)*60)
result_df_encoded.shape
result_df_encoded


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2
252,18,1,9.630476,0.0,0.0,0.0,1.0
1243,18,1,7.958730,0.0,0.0,0.0,1.0
3747,18,1,9.629630,0.0,0.0,0.0,1.0
8039,18,1,9.626517,0.0,0.0,0.0,1.0
8396,18,1,9.629630,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
41969,70,5,99.174603,1.0,0.0,0.0,0.0
13828,70,6,203.142857,0.0,0.0,1.0,0.0
36422,70,6,203.132275,0.0,0.0,1.0,0.0
4495,70,6,119.047619,1.0,0.0,0.0,0.0


In [85]:
X = result_df_encoded.drop('Premium_rate', axis=1).values
y = result_df_encoded['Premium_rate'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled=X_scaled.reshape(-1,10,X.shape[1])
y = y.reshape(-1,10,1)
print(X)
print(X_scaled)
print(y)

[[18.  1.  0.  0.  0.  1.]
 [18.  1.  0.  0.  0.  1.]
 [18.  1.  0.  0.  0.  1.]
 ...
 [70.  6.  0.  0.  1.  0.]
 [70.  6.  1.  0.  0.  0.]
 [70.  6.  1.  0.  0.  0.]]
[[[-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  ...
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]]

 [[-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  ...
  [-1.71508502 -1.50463515 -0.67486187 -0.60248809 -0.47950045
    1.81019795]
  [-1.71508502 -1.50463515 -

In [82]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming 'result_df_encoded' is your preprocessed DataFrame

# Extract features and target variable



# Split data into training and test sets
X_train_org, X_test_org, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features


# Reshape input to be 3D [samples, timesteps, features]
X_train = X_train.reshape((-1,1, X_train.shape[1]))
X_test = X_test.reshape((-1, 1, X_test.shape[1]))

# Define the RNN model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', loss)

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [60]:
y_pred = model.predict(X_test)

# Invert scaling to get actual predicted premium values
X_test_actual = scaler.inverse_transform(X_test.reshape(X_test.shape[0], -1))

y_pred

271/271 [==============================] - 1s 3ms/step


array([[ 2.6347244],
       [ 6.0476704],
       [ 7.5642447],
       ...,
       [ 5.2552323],
       [ 5.017692 ],
       [13.613704 ]], dtype=float32)

In [61]:
X_test_org

array([[19.,  3.,  1.,  0.,  0.,  0.],
       [30.,  5.,  0.,  0.,  1.,  0.],
       [40.,  3.,  0.,  0.,  1.,  0.],
       ...,
       [27.,  3.,  0.,  0.,  0.,  1.],
       [36.,  4.,  1.,  0.,  0.,  0.],
       [39.,  9.,  0.,  0.,  0.,  1.]])

In [62]:
comparison_df = pd.DataFrame(X_test_actual, columns=result_df_encoded.drop('Premium_rate', axis=1).columns)
comparison_df['Predicted_Premium_rate'] = y_pred
comparison_df['Actual_Premium_rate'] = y_test

comparison_df.columns

Index(['Age', 'Duration', 'level_2_Female_NP2', 'level_2_Female_NS2',
       'level_2_Male_NP2', 'level_2_Male_NS2', 'Predicted_Premium_rate',
       'Actual_Premium_rate'],
      dtype='object')

In [63]:
groupby_columns = ['Age', 'Duration', 'level_2_Female_NP2', 'level_2_Female_NS2',
       'level_2_Male_NP2', 'level_2_Male_NS2',
       'Actual_Premium_rate']
average_premiums_base = comparison_df[groupby_columns]
average_premiums= average_premiums_base.groupby(['Age', 'Duration', 'level_2_Female_NP2', 'level_2_Female_NS2',
       'level_2_Male_NP2', 'level_2_Male_NS2'])['Actual_Premium_rate'].mean().reset_index()


In [64]:
average_premiums

,Age,Duration,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,Actual_Premium_rate
0,18.0,1.0,0.0,0.0,0.0,1.0,9.211852
1,18.0,1.0,0.0,0.0,1.0,0.0,8.480423
2,18.0,1.0,0.0,1.0,0.0,0.0,2.598297
3,18.0,1.0,1.0,0.0,0.0,0.0,2.140822
4,18.0,2.0,0.0,0.0,0.0,1.0,4.671323
...,...,...,...,...,...,...,...
1568,70.0,5.0,1.0,0.0,0.0,0.0,223.149206
1569,70.0,6.0,0.0,0.0,1.0,0.0,203.132275
1570,70.0,7.0,0.0,0.0,1.0,0.0,382.027513
1571,70.0,7.0,1.0,0.0,0.0,0.0,142.148148


In [ ]:

# Join with comparison_df
comparison_df_with_avg = comparison_df.merge(average_premiums,
                                              left_on=['Age', 'Duration', 'level_2_Female_NP2', 'level_2_Female_NS2',
       'level_2_Male_NP2', 'level_2_Male_NS2'],
                                              right_on=['Age', 'Duration', 'level_2_Female_NP2', 'level_2_Female_NS2',
       'level_2_Male_NP2', 'level_2_Male_NS2'],
                                              how='left')
comparison_df_with_avg

,Age,Duration,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,Predicted_Premium_rate,Actual_Premium_rate_x,Actual_Premium_rate_y
0,19.0,3.0,1.0,0.0,0.0,0.0,1.692515,3.182540,2.618278
1,30.0,5.0,0.0,0.0,1.0,0.0,5.797672,4.298786,4.467082
2,40.0,3.0,0.0,0.0,1.0,0.0,7.592844,8.730159,7.685714
3,45.0,3.0,0.0,0.0,0.0,1.0,11.971178,10.587302,11.025279
4,35.0,3.0,0.0,1.0,0.0,0.0,4.120199,3.619048,4.353457
...,...,...,...,...,...,...,...,...,...
8652,25.0,9.0,0.0,0.0,0.0,1.0,8.451834,8.644444,8.626655
8653,31.0,1.0,1.0,0.0,0.0,0.0,1.260914,23.777778,5.028220
8654,27.0,3.0,0.0,0.0,0.0,1.0,5.580491,7.111111,5.969821
8655,36.0,4.0,1.0,0.0,0.0,0.0,4.695962,5.140317,5.146381


In [66]:

# Calculate percentage difference
comparison_df['Percentage_Difference'] = (
    abs(comparison_df['Predicted_Premium_rate'] - comparison_df['Actual_Premium_rate']) /
    comparison_df['Actual_Premium_rate']
) * 100

# Sort by percentage difference in descending order
comparison_df = comparison_df.sort_values('Percentage_Difference', ascending=False)

# Display records with largest percentage differences
comparison_df.head(100)

,Age,Duration,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,Predicted_Premium_rate,Actual_Premium_rate,Percentage_Difference
4570,45.0,1.0,1.0,0.0,0.0,0.0,6.286091,0.000000,inf
5320,19.0,1.0,1.0,0.0,0.0,0.0,0.765241,0.000000,inf
7138,40.0,1.0,0.0,1.0,0.0,0.0,4.901785,0.000000,inf
6163,24.0,2.0,0.0,0.0,0.0,1.0,5.923079,0.000000,inf
6169,37.0,2.0,0.0,1.0,0.0,0.0,4.232690,0.000000,inf
...,...,...,...,...,...,...,...,...,...
6412,36.0,4.0,0.0,1.0,0.0,0.0,5.154327,0.428571,1102.676280
6212,39.0,4.0,0.0,1.0,0.0,0.0,6.494898,0.548148,1084.880099
964,24.0,7.0,0.0,0.0,1.0,0.0,8.119798,0.729834,1012.554738
6526,63.0,2.0,0.0,1.0,0.0,0.0,36.855461,3.352381,999.381653


In [68]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

# Assuming 'result_df_encoded' is your preprocessed DataFrame

# Group the data by the relevant features and get the maximum duration for each group
max_durations = result_df_encoded.groupby(['Age', 'level_2_Female_NP2', 'level_2_Female_NS2', 'level_2_Male_NP2', 'level_2_Male_NS2'])['Duration'].max().reset_index()

# Create a dictionary to store the time series data
time_series_data = {}
for _, row in max_durations.iterrows():
    key = (row['Age'], row['level_2_Female_NP2'], row['level_2_Female_NS2'], row['level_2_Male_NP2'], row['level_2_Male_NS2'])
    time_series_data[key] = []

# Populate the time series data
for _, row in result_df_encoded.iterrows():
    key = (row['Age'], row['level_2_Female_NP2'], row['level_2_Female_NS2'], row['level_2_Male_NP2'], row['level_2_Male_NS2'])
    time_series_data[key].append(row['Premium_rate'])

# Pad the time series data with -1 (or any value not present in your data) to handle varying lengths
max_length = max(max_durations['Duration'])
padded_data = []
for key, series in time_series_data.items():
    padded_series = series + [-1] * (max_length - len(series))
    padded_data.append(padded_series)

# Convert the padded data to a NumPy array - Reshape to ensure consistent dimensions
X = np.array(padded_data).reshape(len(padded_data), -1, 1) # Reshape to (samples, timesteps, features)

# Create a mask for the padded values
mask = X != -1

# Scale the features (excluding the masked values)
X_scaled = np.zeros_like(X, dtype=float)
for i in range(X.shape[0]):
    for j in range(X.shape[1]): # Iterate over timesteps
        valid_indices = mask[i, j]
        if valid_indices.any(): # Check if there are any valid values at this timestep
            X_scaled[i, j, valid_indices] = StandardScaler().fit_transform(X[i, j, valid_indices].reshape(-1, 1)).flatten()

# Split data into training and test sets
X_train, X_test, mask_train, mask_test = train_test_split(X_scaled, mask, test_size=0.2, random_state=42)

# Define the RNN model with Masking layer
model = Sequential()
model.add(Masking(mask_value=-1, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, X_test, verbose=0)
print('Test Loss:', loss)

y_pred = model.predict(X_test)

# Invert scaling to get actual predicted premium values (You'll need to adjust this based on how you scaled your data)
# Assuming you have a scaler object named 'scaler' from before
# X_test_actual = scaler.inverse_transform(X_test.reshape(X_test.shape[0], -1))

y_pred

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (212,) + inhomogeneous part.